# Setup

Al iniciar un nuevo repositorio debemos trabajar con un environment dedicado o genérico, con una serie de librerías que debemos conocer correctamente.

En este caso, dejaré el requirements para que trabajemos todos con la misma versión.

Ahora, despacito y buena letra:

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# disable seaborn warnings
# import warnings
# warnings.filterwarnings("ignore")

In [6]:
import os

os.getcwd()

'c:\\Users\\datalab\\AppData\\Local\\Programs\\Microsoft VS Code'

In [9]:
import os
os.getcwd()

'c:\\Users\\datalab\\AppData\\Local\\Programs\\Microsoft VS Code'

In [8]:
df_base = pd.read_csv('..\data\Crime_Data_from_2020_to_Present.csv')

FileNotFoundError: [Errno 2] No such file or directory: '..\\data\\Crime_Data_from_2020_to_Present.csv'

In [ ]:
df_base.shape

In [ ]:
df_base.head()

In [ ]:
""""

Lo primero que debemos hacer es identificar las variables más importantes para nuestro análisis. En este caso, podríamos centrarnos en diferentes perspectivas, como la temporal, 
la geográfica o la tipológica. También podríamos querer hacer un análisis de cuales son los agentes de polícia que más intervienen en los crímenes, o cuales son los crímenes más comunes
que ellos resuelven.

En esta primera aproximación, vamos a centrarnos en las variables temporales y geográficas, ya que nos permitirán tener una visión general de la base de datos
y de los crímenes que se cometen en Los Ángeles.

Como podremos observar, la base de datos cuenta con 28 columnas, entre las cuales destacan principalmente el tipo de crimen -crime_type- y la fecha en la que se cometió -date_occ-.
Otro valor relevante es el de la ubicación -location- o el del área en la que se cometió -area_name-.
"""

df_base.info()

In [ ]:
df_base.describe(include=[np.number])

In [ ]:
"""
Vamos a estandarizar los nombres de variables para trabajar en un entorno más limpio y entendible.
"""
columns = df_base.columns
columns = [column.lower() for column in columns]
columns = [column.replace(' ', '_') for column in columns]
df_base.columns = columns

In [ ]:
"""
A continuación, vamos a eliminar las observaciones que contienen valores nulos en las columnas que consideramos más relevantes para nuestro análisis.
"""
cols_pre_drop = df_base.shape[0]
df_base = df_base.dropna(subset=['crm_cd_desc', 'date_occ', 'location', 'area_name'])
cols_post_drop = df_base.shape[0]

print(f'Número de observaciones eliminadas: {cols_pre_drop - cols_post_drop}')

In [ ]:
"""
Ahora vamos a ver cuales son los crímenes más comunes en Los Ángeles. Para ello, vamos a agrupar los datos por el tipo de crimen y a contar el número de observaciones que hay en cada grupo.
"""

df_base['crm_cd_desc'].value_counts()

In [ ]:
"""
Esta claro que esta froma de presentar los datos no es la más adecuada, ya que hay demasiados tipos de crímenes y no se pueden visualizar de forma clara. Por ello, vamos a seleccionar los 20 crímenes más comunes y a visualizarlos en un gráfico de barras.
"""

top_20_crimes = df_base['crm_cd_desc'].value_counts().head(20)
plt.figure(figsize=(10, 6))

sns.barplot(x=top_20_crimes.values, y=top_20_crimes.index, palette='coolwarm')
plt.xlabel('Number of Crimes')
plt.ylabel('Crime Type')
plt.title('Top 20 Crimes')
plt.show()

In [ ]:
descent_code_dict = {
    'A': 'Other Asian',
    'B': 'Black',
    'C': 'Chinese',
    'D': 'Cambodian',
    'F': 'Filipino',
    'G': 'Guamanian',
    'H': 'Hispanic/Latin/Mexican',
    'I': 'American Indian/Alaskan Native',
    'J': 'Japanese',
    'K': 'Korean',
    'L': 'Laotian',
    'O': 'Other',
    'P': 'Pacific Islander',
    'S': 'Samoan',
    'U': 'Hawaiian',
    'V': 'Vietnamese',
    'W': 'White',
    'X': 'Unknown',
    'Z': 'Asian Indian'
}



In [ ]:
"""
Esto está bien para ver como se distribuye el número de crímenes en Los Ángeles de forma genérica, pero
podemos ver el tipo de crimenes en función del origen de la víctima, por ejemplo.
"""
descent_code_dict = {
    'A': 'Other Asian',
    'B': 'Black',
    'C': 'Chinese',
    'D': 'Cambodian',
    'F': 'Filipino',
    'G': 'Guamanian',
    'H': 'Hispanic/Latin/Mexican',
    'I': 'American Indian/Alaskan Native',
    'J': 'Japanese',
    'K': 'Korean',
    'L': 'Laotian',
    'O': 'Other',
    'P': 'Pacific Islander',
    'S': 'Samoan',
    'U': 'Hawaiian',
    'V': 'Vietnamese',
    'W': 'White',
    'X': 'Unknown',
    'Z': 'Asian Indian'
}

top_10_vict_descent = df_base['vict_descent'].map(descent_code_dict).value_counts().head(10).index

for descent in top_10_vict_descent:
    plt.figure(figsize=(15, 8))
    top_crimes = df_base[df_base['vict_descent'].map(descent_code_dict) == descent]['crm_cd_desc'].value_counts().head(20).reset_index()
    top_crimes.columns = ['crm_cd_desc', 'count']
    sns.barplot(data=top_crimes, x='count', y='crm_cd_desc', hue='count', palette='coolwarm', dodge=False, legend=False)
    plt.xlabel('Number of Crimes')
    plt.ylabel('Crime Code 1')
    plt.title(f'Top 20 Crimes for Victim Descent: {descent}')
    plt.show()

In [ ]:
"""Ahora, vamos a mostrar porcentualmente la diferencia de tipos de crimenes en
funcion del origen de la victima con respecto al tipo de origen más numeroso en los angeles, el blanco."""


white_crimes = df_base[df_base['vict_descent'].map(descent_code_dict) == 'White']['crm_cd_desc'].value_counts(normalize=True).head(20)

# Obtener los 5 tipos de vict_descent más numerosos
top_5_vict_descent = df_base['vict_descent'].map(descent_code_dict).value_counts().head(5).index

# Crear un DataFrame para almacenar las variaciones porcentuales
variation_df = pd.DataFrame()

# Calcular la variación para cada uno de los 5 tipos de vict_descent más numerosos
for descent in top_5_vict_descent:
    if descent != 'White':  # Omitir los hombres blancos
        descent_crimes = df_base[df_base['vict_descent'].map(descent_code_dict) == descent]['crm_cd_desc'].value_counts(normalize=True).head(20)
        common_crimes = white_crimes.index.intersection(descent_crimes.index)
        descent_crimes = descent_crimes[common_crimes]
        white_crimes_common = white_crimes[common_crimes]
        variation = (descent_crimes - white_crimes_common) / white_crimes_common * 100
        variation_df[descent] = variation

# Crear gráficos de barras para cada tipo de vict_descent
for descent in variation_df.columns:
    plt.figure(figsize=(15, 8))
    top_crimes = variation_df[descent].reset_index()
    top_crimes.columns = ['crm_cd_desc', 'variation']
    sns.barplot(data=top_crimes, x='variation', y='crm_cd_desc', hue='crm_cd_desc', palette='coolwarm', dodge=False, legend=False)
    plt.xlabel('Crime Code 1')
    plt.ylabel('Variation Percentage')
    plt.title(f'Variation of Crime Types for Victim Descent: {descent} compared to White')
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
"""Ahora, una vez hemos echado un vistazo a los crímenes más comunes en Los Ángeles, vamos a centrarnos en los posibles sesgos
que pudiesesemos cometer en nuestro análisis. Para ello, vamos a ver qué pasa si filtramos por un área en concreto


West LA es una de las zonas con más homogeneidad demográfica de los Ángeles, 
por lo que es un buen punto de partida para analizar posibles sesgos en nuestro análisis.

Soutwest es una de las zonas con más diversidad demográfica de los Ángeles, 
por lo que es un buen punto de partida para analizar posibles sesgos en nuestro análisis.
"""

In [ ]:
# Filtrar los datos por el área de West LA
west_la_df = df_base[df_base['area_name'] == 'West LA']

# Verificamos qué porcentaje de los crimenes se cometen en West LA
west_la_crimes = west_la_df.shape[0] / df_base.shape[0] * 100
print(f'Percentage of Crimes in West LA: {west_la_crimes:.2f}%')



In [ ]:
"""
El 4.55% de los crimenes se cometen en West LA, por lo que podríamos considerarlo una muestra. Ahora, tenemos que verificar si los crímenes que se cometen en West LA son representativos de los crímenes que se cometen en Los Ángeles en general.
"""

# Calcular la distribución de los tipos de crímenes para West LA
west_la_crimes_total = west_la_df['crm_cd_desc'].value_counts()
west_la_crimes = west_la_crimes_total / west_la_crimes_total.sum()

# Calcular la distribución de los tipos de crímenes para Los Ángeles en general
la_crimes_total = df_base['crm_cd_desc'].value_counts()
la_crimes = la_crimes_total / la_crimes_total.sum()

# Obtener los 20 tipos de crímenes más comunes en West LA
west_la_top_crimes = west_la_crimes.head(20)

# Asegurarse de que solo se consideren los mismos crímenes en ambas zonas
common_crimes = west_la_top_crimes.index.intersection(la_crimes.index)
west_la_top_crimes = west_la_top_crimes[common_crimes]
la_top_crimes = la_crimes[common_crimes]

# Calcular la variación porcentual
variation = (west_la_top_crimes - la_top_crimes) / la_top_crimes * 100
variation = variation.dropna()

# Graficar los 20 tipos de crímenes más comunes en West LA
plt.figure(figsize=(10, 6))
sns.barplot(x=west_la_top_crimes.values, y=west_la_top_crimes.index, palette='coolwarm')
plt.xlabel('Proportion of Crimes')
plt.ylabel('Crime Type')
plt.title('Top 20 Crimes in West LA')
plt.show()

# Graficar la variación porcentual de los tipos de crímenes en West LA comparado con Los Ángeles
plt.figure(figsize=(10, 6))
sns.barplot(x=variation.values, y=variation.index, palette='coolwarm')
plt.xlabel('Variation Percentage')
plt.ylabel('Crime Type')
plt.title('Variation of Crime Types in West LA compared to Los Angeles')
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Convertir la columna date_occ a formato de fecha
df_base['date_occ'] = pd.to_datetime(df_base['date_occ'])

# Extraer el año y el mes de la columna date_occ
df_base['year_month'] = df_base['date_occ'].dt.to_period('M')

#filter by max dec 2023
df_base = df_base[df_base['year_month'] <= '2023-12']

# Contar el número de crímenes por mes
monthly_crimes = df_base['year_month'].value_counts().sort_index()

# Graficar la línea de tiempo de crímenes por mes
plt.figure(figsize=(12, 6))
sns.lineplot(x=monthly_crimes.index.astype(str), y=monthly_crimes.values, marker='o', color='blue')
plt.xlabel('Year-Month')
plt.ylabel('Number of Crimes')
plt.title('Number of Crimes through Time (Monthly)')
plt.xticks(rotation=90)
plt.show()

# La importancia de la verificación de los datos

In [ ]:
df_base.head()

In [ ]:
# ver los estadisticos de la edad de la victima

df_base['vict_age'].describe()

"""
Debemos tener en cuenta que los datos no siempre son perfectos y que pueden contener errores. Por ejemplo, la edad de la víctima no puede ser negativa,
o destacar la excesiva edad de la victima de 120 años.
"""

In [ ]:
# boxplot to vict age
plt.figure(figsize=(10, 6))
sns.boxplot(data=df_base, y='vict_age')
plt.ylabel('Victim Age')
plt.title('Boxplot of Victim Age')


print(r"""El boxplot, o gráfico de velas, es una forma muy útil de visualizar la distribución de una variable numérica. En este caso, podemos ver que la mayoría de las víctimas tienen una edad comprendida entre los 20 y los 40 años, aunque hay algunas excepciones.
    el cuadro azul representa el rango intercuartílico, que es el rango en el que se encuentran el 50% de las observaciones. 
    La línea intermedia del rango intercuartílico es la mediana, que es el valor que divide el conjunto de datos en dos partes iguales.
    Las líneas exteriores del cuadro azul son los cuartiles 1 y 3, que dividen el conjunto de datos en cuatro partes iguales.
    Después tenemos las líneas negras que salen del cuadro, que representan 1,5 veces el rango intercuartílico.
    Por último, los puntos en los extremos representan los valores atípicos o outliers.
""")